In [ ]:
import ivy
import torch
import tensorflow as tf
import numpy as np

In [ ]:
TF_LSTM = ivy.transpile(torch.nn.LSTM, source="torch", target="tensorflow")

torch_lstm = torch.nn.LSTM(2, 2, 1)
tf_lstm = TF_LSTM(2, 2, 1)

x = torch.rand((20, 32, 2))
original_output = torch_lstm(x)
x = tf.constant(x.cpu().numpy())
transpiled_output = tf_lstm(x)

assert np.allclose(original_output[0].detach().cpu().shape, transpiled_output[0].numpy().shape)

/usr/local/lib/python3.10/dist-packages/ivy/utils/exceptions.py:383: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(
